In [133]:
import numpy as np
import sys
# option to import from github folder
# sys.path.insert(0, 'C:/Users/carse/github/suite2p')
import suite2p
from suite2p.run_s2p import run_s2p
import os 
import glob
import admin_functions1 as adfn
from skimage import io
from skimage.external.tifffile import imread, TiffFile 
from PIL import Image
# Jupyter specific autoreloading for external functions (in case changes are made)
%load_ext autoreload
%autoreload
# %matplotlib widget

# Define paths
#---------------------------------------------------------------------------
Fdrop = '/Users/dominicburrows/Dropbox/PhD/analysis/'
Fmac = '/Users/dominicburrows/Documents/'
F10t = '/Volumes/Dominic 10tb/'
Ffig = '/Users/dominicburrows/Dropbox/PhD/figures/'
Ftm = '/Volumes/Dominic Time Machine/'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocess files

In [ ]:
#PUT ALL TIFFS INTO SINGLE FILE TO RUN SUITE2P
numlist = adfn.name_zero('front', 5,5, 1)
for t in numlist:
    
    fish = 'PTZ-WILDTYPE-' + t + '/2photon/sess-01-6dpf/'
    read_path = Ftm + 'Project/' + fish
    save_path = F10t + 'Project/' + fish[:fish.find('2photon')-1]
    os.mkdir(save_path)
        
    for y in range(10):
        os.chdir(read_path)
        blnlist = sorted(glob.glob('*BLN' + '*plane' + str(y) + '*tif'))
        p5list = sorted(glob.glob('*PTZ05' + '*plane' + str(y) + '*tif'))
        p20list = sorted(glob.glob('*PTZ20' + '*plane' + str(y) + '*tif'))
        alllist = np.append(blnlist, np.append(p5list, p20list))
        pid = alllist[0][alllist[0].find('PTZ-WILDTYPE'):alllist[0].find('WILDTYPE')+11] + '_' + alllist[0][alllist[0].find('pla'):alllist[0].find('pla')+6 ]
        os.chdir(save_path)
        os.mkdir(pid)

        for x in range(len(alllist)):
            io.imsave(arr = imread(read_path + alllist[x]),  fname= save_path + '/' + pid + '/' + str(alllist[x]))


In [ ]:
#SAVE REGISTERED TIFFS IN TEMPORARY FOLDER FOR SUITE2P
#loop through registered tiffs and save new folder containing them - to run suite2p
#------------------------------------------------------------------------------------------------------

# Define which fish 
#---------------------------------------------------------------------------
for t in range(3,4):
    
    fish = 'PTZ-WILDTYPE-0' + str(t) + '/2photon/sess-01-6dpf/'
    finalpath = Ftm + 'Project/' + fish
    tmpath = Ftm + 'Project/' + fish[:fish.find('2photon')-1]

    for i in range(3):
        if i == 0:
            condition = 'BLN'
        elif i ==1:
            condition = 'PTZ05'
        elif i ==2:
            condition = 'PTZ20'
        
        for y in range(10):
            os.chdir(finalpath)
            alllist = sorted(glob.glob('*' + condition + '*plane' + str(y) + '*'))
            pid = alllist[0][alllist[0].find('PTZ-WILDTYPE'):alllist[0].find('WILDTYPE')+11] + alllist[0][alllist[0].find('dpf')+3:alllist[0].find('dpf')+8] + alllist[0][alllist[0].find('pla'):alllist[0].find('pla')+6 ]
            os.chdir(Ftm + 'Project/' + fish[:16])
            os.mkdir(pid)

            for x in range(len(alllist)):
                io.imsave(arr = imread(finalpath + alllist[x]),  fname= tmpath + '/' + pid + '/' + str(alllist[x]))

In [ ]:
#REGISTRATION FAILURE on single plane - EXTRACT RAW TIFF FOR PLANE - suite2p without registration
# Loop through raw tiffs to pull out individual plane 
#--------------------------------------------------------------------------------------

# Define which fish 
#---------------------------------------------------------------------------
experiment = 'PTZ-WILDTYPE'
date = '190711-WILDTYPE'
fish = 'F12-PTZ05-WILDTYPE-2photon-6dpf-gc6s-15%-sess-01-run-01'
finalpath = F10t + '/Data2process/Project/' +  experiment + os.sep + date + os.sep + fish
os.chdir(finalpath)
plane = 0

planelist = sorted(glob.glob('*' + str(plane) + '.tif'))
newfold = F10t + '/Data2process/Project/' +  experiment + os.sep + date + os.sep + fish[:18] + 'plane' + str(plane)
os.mkdir(newfold)

for i in range(len(planelist)):
    io.imsave(arr = imread(finalpath + os.sep + planelist[i]),  fname= newfold + os.sep + str(planelist[i]))

## Define suite2p settings

In [135]:
# set your options for running
# overwrites the run_s2p.default_ops
#-----------------------------------------

ops = {
            'fast_disk': [], # used to store temporary binary file, defaults to save_path0 (set as a string NOT a list)
            'save_path0': [], # stores results, defaults to first item in data_path
            'delete_bin': 1, # whether to delete binary file after processing
            # main settings
            'nplanes' : 1, # each tiff has these many planes in sequence
            'nchannels' : 1, # each tiff has these many channels per plane
            'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
            'diameter':5, # def = 5, this is the main parameter for cell detection, 2-dimensional if Y and X are different (e.g. [6 12])
            'tau':  1., # this is the main parameter for deconvolution
            'fs': 2.73,  # sampling rate (total across planes)
            # output settings
            'save_mat': False, # whether to save output as matlab files
            'combined': False, # combine multiple planes into a single result /single canvas for GUI
            # parallel settings
            'num_workers': 0, # 0 to select num_cores, -1 to disable parallelism, N to enforce value
            'num_workers_roi': 2, # 0 to select number of planes, -1 to disable parallelism, N to enforce value
            # registration settings
            'do_registration': True, # whether to register data
            'nimg_init': 200, #def = 200, subsampled frames for finding reference image
            'batch_size': 200, # number of frames per batch
            'maxregshift': 0.05, # max allowed registration shift, as a fraction of frame max(width and height)
            'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
            'reg_tif': 1, # whether to save registered tiffs
            'subpixel' : 10, # precision of subpixel registration (1/subpixel steps)
            # cell detection settings
            'connected': True, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
            'navg_frames_svd': 5000, # max number of binned frames for the SVD
            'nsvd_for_roi': 1000, # max number of SVD components to keep for ROI detection
            'max_iterations': 20, # maximum number of iterations to do cell detection
            'ratio_neuropil': 6.0, # ratio between neuropil basis size and cell radius
            'ratio_neuropil_to_cell': 3, # minimum ratio between neuropil radius and cell radius
            'tile_factor': 1, # use finer (>1) or coarser (<1) tiles for neuropil estimation during cell detection
            'threshold_scaling': 1., # adjust the automatically determined threshold by this scalar multiplier
            'max_overlap': 0.75, # cells with more overlap than this get removed during triage, before refinement
            'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
            'outer_neuropil_radius': np.inf, # maximum neuropil radius
            'min_neuropil_pixels':350, # minimum number of pixels in the neuropil
            # deconvolution settings
            'baseline': 'maximin', # baselining mode
            'win_baseline': 60., # window for maximin
            'sig_baseline': 10., # smoothing constant for gaussian filter
            'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
            'neucoeff': .7,  # neuropil coefficient
          }
        

## Run suite2p

In [ ]:
# provide an h5 path in 'h5py' or a tiff path in 'data_path'
# db overwrites any ops (allows for experiment specific settings)
# db = {
#       'h5py': [], # a single h5 file path
#       'h5py_key': 'data',
#       'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
#       'data_path': ['C:/Users/carse/tiffs'], # a list of folders with tiffs 
#                                              # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)
#       'subfolders': [], # choose subfolders of 'data_path' to look in (optional)
#       'fast_disk': 'C:/BIN', # string which specifies where the binary file will be stored (should be an SSD)
#     }

# run one experiment
# opsEnd=run_s2p(ops=ops,db=db)

In [136]:
# make a list of databases and loop over them
#---------------------------------------------
db = []

numlist = adfn.name_zero('front', 5,5, 1)
for i in numlist:
    fish = 'PTZ-WILDTYPE-' + i
    read_path = F10t + 'Project/' + fish + '/'
    for y in range(10):
        plane = 'PTZ-WILDTYPE-' + str(i) +  '_' + 'plane' + str(y)
        final_path = read_path + plane
        os.chdir(final_path)
        db.append({'data_path': [final_path]})

In [137]:
db

[{'data_path': ['/Volumes/Dominic 10tb/Project/PTZ-WILDTYPE-05/PTZ-WILDTYPE-05_plane0']},
 {'data_path': ['/Volumes/Dominic 10tb/Project/PTZ-WILDTYPE-05/PTZ-WILDTYPE-05_plane1']},
 {'data_path': ['/Volumes/Dominic 10tb/Project/PTZ-WILDTYPE-05/PTZ-WILDTYPE-05_plane2']},
 {'data_path': ['/Volumes/Dominic 10tb/Project/PTZ-WILDTYPE-05/PTZ-WILDTYPE-05_plane3']},
 {'data_path': ['/Volumes/Dominic 10tb/Project/PTZ-WILDTYPE-05/PTZ-WILDTYPE-05_plane4']},
 {'data_path': ['/Volumes/Dominic 10tb/Project/PTZ-WILDTYPE-05/PTZ-WILDTYPE-05_plane5']},
 {'data_path': ['/Volumes/Dominic 10tb/Project/PTZ-WILDTYPE-05/PTZ-WILDTYPE-05_plane6']},
 {'data_path': ['/Volumes/Dominic 10tb/Project/PTZ-WILDTYPE-05/PTZ-WILDTYPE-05_plane7']},
 {'data_path': ['/Volumes/Dominic 10tb/Project/PTZ-WILDTYPE-05/PTZ-WILDTYPE-05_plane8']},
 {'data_path': ['/Volumes/Dominic 10tb/Project/PTZ-WILDTYPE-05/PTZ-WILDTYPE-05_plane9']}]

In [ ]:
#RUN
#-----------------
#Fme =  F10t + '/Data2process/Project/' +  experiment + os.sep + date + os.sep + fish[:18] + 'plane' + str(plane)
#db = []
#db.append({'data_path': [Fme]})
                        
for dbi in db:
    opsEnd=run_s2p(ops=ops,db=dbi)

{'data_path': ['/Volumes/Dominic 10tb/Project/PTZ-WILDTYPE-05/PTZ-WILDTYPE-05_plane0']}
tif
** Found 150 tifs - converting to binary **
800 frames of binary, time 9.65 sec.
1600 frames of binary, time 19.21 sec.
2400 frames of binary, time 27.27 sec.
3200 frames of binary, time 34.75 sec.
4000 frames of binary, time 44.53 sec.
4800 frames of binary, time 52.54 sec.
5600 frames of binary, time 57.76 sec.
6400 frames of binary, time 61.71 sec.
7200 frames of binary, time 70.02 sec.
8000 frames of binary, time 79.94 sec.
8800 frames of binary, time 90.27 sec.
9600 frames of binary, time 100.38 sec.
time 356.72 sec. Wrote 29484 tiff frames to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
registering 29484 frames
Reference frame, 4.89 sec.
1000/29484 frames, 28.48 sec.
2000/29484 frames, 57.72 sec.
3000/29484 frames, 87.42 sec.
4000/29484 frames, 116.43 sec.
5000/29484 frames, 145.46 sec.
6000/29484 frames, 175.06 sec.
7000/29484 frames,

## Backup data

In [ ]:
#BACKUP NONREGISTERED TIFFS IN SUITE2P OUTPUT FORMAT
#After doing suite2p on single plane that failed registration, can back up these tiffs 
#in same format as suite2p output
#-----------------------------------------------------------------------------------
experiment = 'PTZ-WILDTYPE'
date = '190711-WILDTYPE'
fish = 'F12-PTZ05-WILDTYPE-2photon-6dpf-gc6s-15%-sess-01-run-01-suite2p'
finalpath = F10t + '/Data2process/Project/' +  experiment + os.sep + date + os.sep + fish + os.sep + 'plane9/reg_tif/'
fonalpath = F10t + '/Data2process/Project/' +  experiment + os.sep + date + os.sep + fish + os.sep + 'plane9/reg_tif*/'
os.chdir(finalpath)
os.mkdir(fonalpath)
tiflist = sorted(glob.glob('*.tif'))
counter = 0

for i in range(50):
    omlist = []
    
    if i == 49:
        for y in range(28):
            omlist.append(Image.fromarray(io.imread(finalpath + tiflist[counter])))
            counter+=1
            
        omlist[0].save(fonalpath + "file_chan049.tif", save_all=True,
               append_images=omlist[1:])
    
    for y in range(200):
        omlist.append(Image.fromarray(io.imread(finalpath + tiflist[counter])))
        counter+=1
    
    if i < 10:
        omlist[0].save(fonalpath + "file_chan00" + str(i) + ".tif", save_all=True,
               append_images=omlist[1:])
    elif i > 9:
        omlist[0].save(fonalpath + "file_chan0" + str(i) + ".tif", save_all=True,
               append_images=omlist[1:])
        

In [20]:
#BACKUP OPS files from raw suite2p data
#--------------------------------------------------------------------------------------
experiment = 'PTZ-WILDTYPE-06'
savepath = Ftm + 'Project/' + experiment + '/2photon/sess-01-6dpf/'

for i in range(3):
    if i == 0:
        condition = 'BLN_'
        candition = 'BLN'
    elif i ==1:
        condition = 'PTZ0'
        candition = 'PTZ05'
    elif i ==2:
        condition = 'PTZ2'
        candition = 'PTZ20'

               
    for y in range(10):
        saveops = np.load(Ftm + 'Project/' + experiment + os.sep + experiment + '_' + condition + 'plane' + str(y) + os.sep + 'suite2p/plane0/ops.npy')
        savestat = np.load(Ftm + 'Project/' + experiment + os.sep + experiment + '_' + condition + 'plane' + str(y) + os.sep + 'suite2p/plane0/stat.npy')
        np.save(savepath + experiment + '_2photon_sess-01-6dpf_' + candition + '_run-01_plane' + str(y) + '_ops.npy', saveops)
        np.save(savepath + experiment + '_2photon_sess-01-6dpf_' + candition + '_run-01_plane' + str(y) + '_stat.npy', savestat)